In [2]:
import pandas as pd
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
import math

### Lectura del fichero CSV

In [44]:
data = pd.read_csv('Datos_Montabilidad_Anom_447-6-1.csv',delimiter=';')

# Filtrado y construcción del dataframe binario
df_binario = data.iloc[:, 2:-2].apply(lambda x: x > 0).astype(int)

# Data frame con las primeras 2 filas y 2 columnas
np_binario_0_20 = df_binario.iloc[:2, 4:5].to_numpy()

print(np_binario_0_20)
#Separar por vectores train y test
##train_data = data[data['Set'] == 'Train']
##test_data = data[data['Set'] == 'Test']

[[1]
 [0]]


### Implementación de SWAP Test (2 vectores N columnas)
***Cuando subamos la cantidad de vectores hay que replantear cuantos wires (qbits) utilizar.*

Se calculan los qubits necesarios a partir de la dimensionalidad del vector (cantidad de columnas). El vector se define como:

$\mathbf{x} = (x_1, x_2, x_3, \ldots, x_n)$


Los qubits que vamos a necesitar para codificar (BasisEmbedding) son $ qubits = dim(\mathbf{x})$, ademas:
* Para cada dimension necesitamos un qbits ancilla
* para cada vector necesitamos un qubit por dimentsion

Si tenemos N vectores necesitaremos multiplicar N+1 veces los qubits requeridos para codificar N por cada vector y +1 para los qubits ancilla.



In [45]:
#Numero maximo de quibts que puede comparar son 8
N_vectores, qbit_number = np_binario_0_20.shape
qubits_totales = 3* qbit_number

estados_concatenados =np_binario_0_20.ravel().tolist()
print(estados_concatenados)

[1, 0]


In [46]:
dev = qml.device("default.qubit", qubits_totales, shots =10)
#dev_qiskit = qml.device("qiskit.aer", qubits_totales)
@qml.qnode(dev)
def circuit(estados_concatenados):
    qml.BasisEmbedding(features=estados_concatenados, wires=range(qbit_number, qubits_totales))
    ancillea = []
    for i in range(qbit_number):
        qml.Hadamard(i)
        anc = i
        ancillea.append(anc)
        first_state = i+qbit_number
        second_state = i+ 2*qbit_number
        qml.CSWAP(wires=[anc, first_state, second_state])
        qml.Hadamard(i)
    return  qml.expval(qml.operation.Tensor(*[qml.PauliZ(i) for i in ancillea]))
    
    #return qml.expval(qml.operation.Tensor(*[qml.PauliZ(i) for i in ancillea]))
    #qml.probs(wires = range(len(ancillea)))

estado_0 = circuit(estados_concatenados)
#matriz_estados_propios_salida = estado_0.numpy().
#matriz_salida_prob_0 = [round(matriz_probabilidades_salida[0],1)]
# Crear un gráfico de línea
#plt.plot(matriz_salida_prob_0)
# Mostrar el gráfico
#plt.show()
#dev._circuit.draw(output="mpl")
print(qml.draw(circuit)(estados_concatenados))
print(estado_0)

0: ──H──────────────╭●─────H─┤  <Z>
1: ─╭BasisEmbedding─├SWAP────┤     
2: ─╰BasisEmbedding─╰SWAP────┤     
0.0
